In [6]:
import json
import facebook
import requests
import pandas as pd
import sqlite3
from config.config import SQL_SCHEMA, FACEBOOK_ACCESS_TOKEN, FACEBOOK_PAGE_ID

video_id = ''
video_list = []
video_insights_list = []
graph = facebook.GraphAPI(FACEBOOK_ACCESS_TOKEN)
print('here')
videos = graph.get_all_connections(id=FACEBOOK_PAGE_ID, connection_name='videos', fields='id, title, description, created_time')
print(videos)
for video in videos:
    video_list.append(video)
print(video_list)    

here
<generator object GraphAPI.get_all_connections at 0x7ffa0020dd60>
[{'id': '352728366115496', 'created_time': '2020-08-21T13:59:47+0000'}, {'id': '1234138873601756', 'title': 'ReThink: What is Eternal Life?', 'description': '4 Common Misconceptions about Eternal Life', 'created_time': '2020-08-16T12:45:22+0000'}, {'id': '683163315876752', 'created_time': '2020-08-14T13:53:42+0000'}, {'id': '875196306340063', 'title': 'ReThink: Who is the Holy Spirit?', 'description': 'The sermon will be focused on the relational aspect of knowing the Spirit, walking with the Spirit, being filled, and partnering missionally with the third member of the Trinity.', 'created_time': '2020-08-09T12:45:15+0000'}, {'id': '3300673073313007', 'title': 'ReThingk: What is the Church?', 'created_time': '2020-08-02T12:47:46+0000'}, {'id': '981518438942902', 'created_time': '2020-07-31T13:51:06+0000'}, {'id': '671758730088154', 'description': 'Worship Night in the parking lot behind the church!!', 'created_time':

In [2]:
for video in video_list:
        video_insights = graph.get_connections(id=video['id'],
                                               connection_name='video_insights',
                                               metric='total_video_views, total_video_stories_by_action_type',
                                               fields='values, id')
        video_insights_list.append(video_insights)

In [3]:
facebook_video_list = []
for video in zip(video_list, video_insights_list):
    facebook_video_list.append(video)

In [4]:
facebook_df = pd.DataFrame(index=range(len(video_list)), columns=SQL_SCHEMA)
for c in range(len(facebook_video_list)):
    facebook_df['id'][c] = facebook_video_list[c][0]['id']
    try:
        facebook_df['title'][c] = facebook_video_list[c][0]['title']
    except:
        pass
    try:
        facebook_df['description'][c] = facebook_video_list[c][0]['description']
    except:
        pass
    facebook_df['date_published'][c] = facebook_video_list[c][0]['created_time']
    try:
        facebook_df['views'][c] = facebook_video_list[c][1]['data'][0]['values'][0]['value']
    except:
        pass
    try:
        facebook_df['likes'][c] = facebook_video_list[c][1]['data'][1]['values'][0]['value']['like']
    except:
        pass
    try:
        facebook_df['comments'][c] = facebook_video_list[c][1]['data'][1]['values'][0]['value']['comment']
    except:
        pass
    facebook_df['medium'][c] = 'facebook'

In [5]:
conn = sqlite3.connect('analytics.db')
c = conn.cursor()
c.execute("""DELETE FROM media_stats
    WHERE medium='facebook'""")
# test
facebook_df.to_csv('try.csv', index=False)
facebook_df.to_sql('media_stats', con=conn, if_exists='append', index=False)
test_df = pd.read_sql('SELECT * FROM media_stats', conn)
print(len(test_df), test_df.head())
print('sqlite database appended')

297             id                                        title  \
0  IAmG1eCH0fg               ReThink: What is Eternal Life?   
1  c6vAXVTRvnA             ReThink: Who is the Holy Spirit?   
2  uCzLJtz7Wuo                 ReThink: What is the Church?   
3  wquf54O3JHQ  ReThink: What is the Gospel of the Kingdom?   
4  rV9_xho4PJs              ReThink: How Do I Follow Jesus?   

                                         description        date_published  \
0         4 Common Misconceptions about Eternal Life  2020-08-17T02:15:08Z   
1  The sermon will be focused on the relational a...  2020-08-10T02:18:24Z   
2                                                     2020-08-02T15:34:32Z   
3                                                     2020-07-27T01:57:16Z   
4                                                     2020-07-19T15:16:01Z   

   views  likes  comments   medium  
0  128.0    2.0       0.0  youtube  
1  147.0    2.0       0.0  youtube  
2  113.0    4.0       1.0  youtube  
